In [1]:
from ultralytics import YOLO
import yaml
import os

In [2]:
os.chdir("Hands")

FileNotFoundError: [Errno 2] No such file or directory: 'Hands'

In [ ]:
import yaml
import os
from pathlib import Path

# Get absolute base path
BASE_PATH = Path.cwd()  # Current working directory

# List your dataset folders
dataset_folders = [
    'ASL_roboflow',
    'Bharatanatyam_Mudras',
    'Hand_Gesture_roboflow',
    'synthetic',
    'yolo_hands_kaggle'
]

print("="*60)
print("READING DATASETS AND MERGING CLASSES")
print("="*60)

all_classes = []
valid_datasets = []

for folder in dataset_folders:
    yaml_path = Path(folder) / 'data.yaml'
    if yaml_path.exists():
        try:
            with open(yaml_path, 'r') as f:
                data = yaml.safe_load(f)
                names = data.get('names', [])
                
                # Check if train/valid folders actually exist
                train_path = Path(folder) / 'train'
                valid_path = Path(folder) / 'valid'
                
                if train_path.exists() and valid_path.exists():
                    print(f"✅ {folder}: {len(names)} classes")
                    print(f"   Classes: {names[:5]}...")
                    all_classes.extend(names)
                    valid_datasets.append(folder)
                else:
                    print(f"⚠️  {folder}: train/valid folders not found")
        except Exception as e:
            print(f"❌ {folder}: Error - {e}")
    else:
        print(f"❌ {folder}: data.yaml not found")

# Remove duplicate classes while preserving order
unique_classes = list(dict.fromkeys(all_classes))
print(f"\n📊 Total unique classes: {len(unique_classes)}")
print(f"   All classes: {unique_classes}")

# Create multi-dataset config with ABSOLUTE paths
multi_config = {
    'path': str(BASE_PATH),  # Absolute base path
    'train': [str(Path(folder) / 'train' / 'images') for folder in valid_datasets],
    'val': [str(Path(folder) / 'valid' / 'images') for folder in valid_datasets],
    'nc': len(unique_classes),
    'names': unique_classes
}

# Save the unified config
output_file = 'multi_dataset.yaml'
with open(output_file, 'w') as f:
    yaml.dump(multi_config, f, default_flow_style=False, sort_keys=False)

print(f"\n✅ Created {output_file}")
print("\n" + "="*60)
print("YAML CONTENT:")
print("="*60)
with open(output_file, 'r') as f:
    print(f.read())

# Now train
print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

from ultralytics import YOLO
import torch

# Check device
print(f"MPS available: {torch.backends.mps.is_available()}")

# Initialize model from scratch
model = YOLO('yolov8n.yaml')  # Architecture only, no weights

# Train
results = model.train(
    data=output_file,
    epochs=50,
    imgsz=416,  # ~40% faster, slight accuracy drop
    batch=32,
    name='yolo_from_scratch_multi',
    pretrained=False,
    patience=50,
    device='mps',  # Use Apple Silicon GPU
    workers=4,
    
    # Training from scratch parameters
    warmup_epochs=5,
    lr0=0.01,
    lrf=0.01,
    
    # Data augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
)

print("\n✅ Training complete!")
print(f"Results saved at: {results.save_dir}")